In [1]:
from pls_es import *
nb_objectif = 2
file = "2KP50-TA-0.dat"
w = (0.2,0.8)
nb_objet = 20
# w = [0.2,0.1,0.2,0.3,0.15,0.05]
infos = read_file(file,nb_objectif,nb_objet)


In [2]:
print("Solu_random:",get_v_total(infos,init_p(infos)[0],w = w,model = 'ponderee'))

Solu_random: 4505.6


Évaluer à l’aide de plusieurs essais le nombre de questions nécessaires pour identifier la solution optimale pour le décideur.

In [3]:
pareto_index = PLS(file,nb_objectif = nb_objectif,nb_objet =nb_objet)
print(len(pareto_index)," solutions")
for s in pareto_index:
    # print(s)
    # print(get_v_total(infos,s,w = (0.2,0.8),model = 'ponderee'))
    # print(get_v_total(infos,s,w = (0.2,0.8),model = 'OWA'))
    pass

Trouver voisins: 14
Trouver voisins: 29
Trouver voisins: 32
Trouver voisins: 16
Trouver voisins: 44
Trouver voisins: 19
Trouver voisins: 18
Trouver voisins: 33
Trouver voisins: 17
Trouver voisins: 16
Trouver voisins: 13
Trouver voisins: 19
Trouver voisins: 17
Trouver voisins: 32
Trouver voisins: 17
Trouver voisins: 17
Trouver voisins: 29
Trouver voisins: 17
Trouver voisins: 20
Trouver voisins: 16
Trouver voisins: 16
Trouver voisins: 17
Trouver voisins: 18
Trouver voisins: 16
Trouver voisins: 18
Trouver voisins: 17
9  solutions


In [4]:
pareto_index = procedure1(infos,pareto_index,w,'ponderee')
print("Procedure2:",get_v_total(infos,pareto_index,w = w,model = 'ponderee'))

TypeError: cannot unpack non-iterable NoneType object

In [8]:
pareto_index = PLS_EI(w,'ponderee',file,nb_objectif = nb_objectif)
print("Procedure2:",get_v_total(infos,pareto_index,w = w,model = 'ponderee'))

Procedure2: 15567.600000000002


In [9]:
res = procedure12(infos,pareto_index,w = w,model = 'ponderee')
print("Procedure2:",get_v_total(infos,res,w = w,model = 'ponderee'))

AttributeError: 'dict' object has no attribute 'shape'

In [10]:
pareto_index = PLS_EI(w,'OWA',file,nb_objectif = nb_objectif)
print("Procedure2:",get_v_total(infos,pareto_index,w = w,model = 'OWA'))

Procedure2: 15730.0


In [11]:
import gurobipy as gp
def PMROWA(x, y, prefs, verbose = False):

    n = x.shape[0]

    #OWA
    x.sort()
    y.sort()

    #creation du model
    model = gp.Model("MMR")
    #creation des variables w
    w = [model.addVar(vtype=GRB.CONTINUOUS, name="w"+str(i+1)) for i in range(n)]

    model.update()

    if not verbose:
        model.Params.LogToConsole = 0

    #Définition des contraintes

    #contrainte 1
    for i in range(n):
        model.addConstr(w[i]>=0)

    #contrainte 2
    for i in range(n-1):
        model.addConstr(w[i] >= w[i+1])

    #contrainte 3
    model.addConstr(np.sum(w) == 1)

    #contrainte 4
    for a, b in prefs:
        #OWA
        sa = np.sort(a)
        sb = np.sort(b)

        model.addConstr(np.sum([np.sum(w[i])*sa[i] for i in range(n)]) >= np.sum([np.sum(w[i])*sb[i] for i in range(n)]))

    #Définition de l'objectif
    obj = gp.LinExpr()
    obj = 0

    obj = np.sum([ w[i] * y[i] for i in range(n)]) - np.sum( [w[i] * x[i] for i in range(n)])

    model.setObjective(obj,GRB.MAXIMIZE)

    #Run l'optimiseur
    model.optimize()

    return obj.getValue()

In [12]:
def MR(x, front, prefs):

    nb_solution = front.shape[0]

    max_value = float('-inf')
    max_index = -1

    for y_index in range(nb_solution):

        if y_index == x:
            continue

        value = PMR_OWA(front[x], front[y_index], prefs)

        if value > max_value:
            max_value = value
            max_index = y_index

    return max_index, max_value

In [13]:
def MMR(front, prefs):

    nb_solution = front.shape[0]

    min_value = float('inf')
    min_x_index = -1
    min_y_index = -1

    for x_index in tqdm(range(nb_solution)):

        y_index, y_value = MR(x_index, front, prefs)

        if y_value < min_value:
            min_value = y_value
            min_x_index = x_index
            min_y_index = y_index

    return min_x_index, min_y_index, min_value